In [2]:
import os
import numpy as np 
import pandas as pd 

In [ ]:
!wget -P ./../../../dataset/ -nc https://cleanlab-public.s3.amazonaws.com/RegressionBenchmark/telomere_cq_telo.csv

In [4]:
path = "./../../../dataset/telomere_cq_telo.csv"
dataset_name = os.path.splitext(os.path.basename(path))[0]
data = pd.read_csv(path, index_col=0)
print("shape of data: ", data.shape)
data.head() 

shape of data:  (10000, 5)


,true_error,true.dna.scg,true.telo.var,true_label,given_label
0,0,8.577178,1.033903,14.885612,14.902281
1,0,10.425544,0.992451,14.663097,14.695333
2,0,10.635183,1.008781,14.610830,14.638863
3,0,8.879612,0.957411,14.946508,14.910670
4,0,8.319504,1.007479,14.966969,14.965843


In [5]:
training_data = data.drop(['true_label', 'true_error'], axis=1)
training_data.head(2)

,true.dna.scg,true.telo.var,given_label
0,8.577178,1.033903,14.902281
1,10.425544,0.992451,14.695333


In [6]:
y = training_data['given_label']
X = training_data.drop(['given_label'], axis=1)

###### Add your training code here ###############
# model = 
# model_fit = model.fit(X,y)
# predictions = model.predict(X)

##### saving the predictions ####################
# savepath = "./../predictions/model_predictions.npy"
# np.save(savepath, np.array(predictions))

## Section section is relevant only if using AutoGluon

Uncomment the code in following cells and run to generate output and save to predictions folder. 

### set up arguments 

In [7]:
from autogluon.tabular import TabularPredictor

data_without_label = training_data.drop(['given_label'], axis=1)
savepath = "./../trained_models/"
if not os.path.isdir(savepath):
    os.mkdir(savepath)

predictorArgs = {
                    "path"             : savepath, 
                    "label"            : 'given_label',
                    "eval_metric"      : 'r2',
                    "problem_type"     : 'regression'
                }

hyperparameters = { 
                    'GBM'   : {}, 
                    'FASTAI': {}, 
                    'RF'    : {'criterion': 'squared_error', 
                                'ag_args': {'name_suffix': 'MSE', 
                                'problem_types': ['regression', 'quantile']}}
                    }

### Normal Model fitting

In [8]:
predictor = TabularPredictor(**predictorArgs)
predictor.fit(training_data, hyperparameters=hyperparameters)
leaderboard = predictor.leaderboard(training_data, silent=True)
leaderboard

Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    10000
Train Data Columns: 2
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    2903.44 MB
	Train Data (Original)  Memory Usage: 0.16 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 2 | ['true.dna.scg', 'true.telo.var']
	Types of features in process

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE,0.984628,0.926707,0.189573,0.041053,1.027969,0.189573,0.041053,1.027969,1,True,2
1,LightGBM,0.948620,0.937001,0.013441,0.004060,3.532742,0.013441,0.004060,3.532742,1,True,1
2,NeuralNetFastAI,0.945206,0.940852,0.062581,0.008800,6.224073,0.062581,0.008800,6.224073,1,True,3
3,WeightedEnsemble_L2,0.945206,0.940852,0.074944,0.008956,6.252369,0.012363,0.000156,0.028296,2,True,4


### Cross Validation fitting

In [9]:
predictor_cv = TabularPredictor(**predictorArgs) 
predictor_cv.fit(training_data, presets="best_quality", num_stack_levels= 0, hyperparameters=hyperparameters)
leaderboard_cv = predictor_cv.leaderboard(training_data, silent=True)
leaderboard_cv

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    10000
Train Data Columns: 2
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    2732.95 MB
	Train Data (Original)  Memory Usage: 0.16 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in o

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE_BAG_L1,0.990824,0.932801,0.333788,0.284784,1.101093,0.333788,0.284784,1.101093,1,True,2
1,LightGBM_BAG_L1,0.948749,0.940594,0.122649,0.026261,12.170735,0.122649,0.026261,12.170735,1,True,1
2,WeightedEnsemble_L2,0.947183,0.944740,1.223614,0.387104,55.680739,0.003168,0.000369,0.080766,2,True,4
3,NeuralNetFastAI_BAG_L1,0.945346,0.944710,0.764009,0.075690,42.328145,0.764009,0.075690,42.328145,1,True,3


### predicting and saving arrays 

In [ ]:
models = {  'normal'            : list(leaderboard['model']), 
            'crossValidation'   : list(np.append(leaderboard_cv['model'], ['oof']))}

predictors = { 'normal'          : predictor, 
               'crossValidation' : predictor_cv}

# check if path is available, else create it. 
pred_path = "./../predictions/"
if not os.path.isdir(pred_path):
    os.mkdir(pred_path)

for model_type in models.keys():
    if model_type == "normal": 
        for model in models[model_type]:
            savepath = os.path.join(pred_path, model+".npy")
            predictions = predictors[model_type].predict(data_without_label, model=model)
            np.save(savepath, np.array(predictions))
    elif model_type == "crossValidation":
        for model in models[model_type]:
            if model == "oof":
                predictions = predictors[model_type].get_oof_pred()
                savepath = os.path.join(pred_path, model+".npy")
            elif model in models['normal']:
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+"_CV.npy")
            else: 
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+".npy")

            np.save(savepath, np.array(predictions))